# Jupyter Notebook UI to analyze baseline data from tap-habituation experiments!

Version 1.3 - Joseph Liang, Rankin Lab
Updated:
1. Upgraded folder path selection application
2. Upgraded dataset management (less moving parts for end-user)
3. output changed from tif -> png

## Known bug: Step 2 an empty windows displays in Mac. May also apply to linux/windows.

### Beginner Essentials:
1. Shift-Enter to run each cell. After you run, you should see an output "done step #". If not, an error has occured
2. When inputting your own code/revising the code, make sure you close all your quotation marks '' and brackets (), [], {}.
3. Don't leave any commas (,) hanging! (make sure an object always follows a comma. If there is nothing after a comma, remove the comma!
4. Learning to code? Each line of code is annotated to help you understand how this code works!

**Run all cells/steps sequentially, even the ones that do not need input**

# 1. Importing Packages Required (No input required, just run)

In [1]:
import pandas as pd #<- package used to import and organize data
import numpy as np #<- package used to import and organize data
import seaborn as sns #<- package used to plot graphs
from matplotlib import pyplot as plt #<- package used to plot graphs
import os #<- package used to work with system filepaths
from ipywidgets import widgets #<- widget tool to generate button
from IPython.display import display #<- displays button
from ipyfilechooser import FileChooser
# from tkinter import Tk, filedialog #<- Tkinter is a GUI package
from tqdm.notebook import tqdm
# import dask.dataframe as dd
# import pingouin as pg
pd.set_option('display.max_columns', 50)
print("done step 1")

done step 1


## 2. Pick filepath (just run and click button from output)

Run the following cell and click the button 'Select Folder' to pick a filepath.

**Important: Later on, this script uses the total file path for each file to import and group data. That means if your folder has whatever your strain is named, the script will not work.**

(ex. if your folder has "N2" in it this script sees all files inside this folder as having the "N2" search key)

**An easy fix is to just rename your folder to something else (make your strains lower-case, or just have the date)**

In [2]:
starting_directory = '/Users'
chooser = FileChooser(starting_directory)
display(chooser)

FileChooser(path='/Users', filename='', title='', show_hidden=False, select_desc='Select', change_desc='Change…

In [3]:
print(chooser.selected_path)
folder_path=chooser.selected_path

/Users/gurmehak/Documents/RankinLab/Test_Datasets/PDScreen_TapHab_August15_2022


In [4]:
screens = ['PD_Screen', 'ASD_Screen', 'G-Proteins_Screen', 'Glia_Genes_Screen', 'Neuron_Genes_Screen']

screen_chooser = widgets.Select(options=screens, value=screens[0], description='Screen:')
display(screen_chooser)

Select(description='Screen:', options=('PD_Screen', 'ASD_Screen', 'G-Proteins_Screen', 'Glia_Genes_Screen', 'N…

In [5]:
Screen=screen_chooser.value

# 3. User Defined Variables (Add input here)

Here, we add some constants to help you blaze through this code.

3.1: Setting time bins


3.2: Setting view range for your graph
- Top, bottom = y axis view range
- left, right = x axis view range



In [6]:
# Setting 1s Bins
bins = np.linspace(0,1200,1201) # np.linspace(start, end, steps in between)
print(bins)


print("done step 3")

[0.000e+00 1.000e+00 2.000e+00 ... 1.198e+03 1.199e+03 1.200e+03]
done step 3


# 4. Construct filelist from folder path (No input required, just run)

In [24]:
os.chdir(folder_path) # setting your working directory so that your images will be saved here

filelist = list() # empty list
for root, dirs, files in os.walk(folder_path): # this for loop goes through your folder 
    for name in files:
        if name.endswith('.dat'): # and takes out all files with a .dat (file that contains your data)
            if "_" in name.split(".")[-2]:
                filepath = os.path.join(root, name) # Notes down the file path of each data file
                filelist.append(filepath) # saves it into the list

if not filelist:
    raise FileNotFoundError("No .dat files found in the selected folder!")
else:
    print(f"Number of .dat files to process: {len(filelist)}")
    # print(f"Example of first and last file saved: {filelist[0]}, {filelist[-1]}") 

print('done step 4')

Number of .dat files to process: 13
done step 4


# 5. Process Data Function (No input required, just run)

In [29]:
def ProcessData(strain): 
    """
    Filters and processes .dat files matching the given strain.

    Parameters: 
        strain (str): keyword to match in the files

    Returns:
        dict: N (Plate number) and Dataframe with required columns ("time", "dura", "dist", "prob", "speed", "plate", "Date","Plate_id","Screen")

    """
    strain_filelist = [x for x in filelist if strain in x] # Goes through the list and filters for keyword
    Strain_N = len(strain_filelist) # Finds the number of plates per strain
    if Strain_N == 0:
        raise AssertionError ('{} is not a good identifier'.format(strain))
    else:
        pass
        print(f'Strain {strain}')
        print(f'Number of plates: {Strain_N}') 
        
        # visiting files in this strain
        strain_filelist = [file for file in filelist if strain in file]
        df_list=[]
        for file in strain_filelist:
            if file.split('/')[-1].startswith('._'):
                pass
            else:
                try:
                    print(f"File: {file}")
                    df= pd.read_csv(file, sep=' ', header = None, encoding_errors='ignore')
                    df['Plate_id'] = file.split('/')[-1].split('_')[-1].split('.')[0]
                    df['Date'] = file.split('/')[-2].split('_')[0]
                    df['Screen'] = file.split('/')[-4]
                    df_list.append(df)
                except:
                    print(f"error in file {file}")
                    pass
        DF_Total = pd.concat(df_list, ignore_index = True)
        DF_Total = DF_Total.rename( 
                    {0:'Time',
                    1:'n',
                    2:'Number',
                    3:'Instantaneous Speed',
                    4:'Interval Speed',
                    5:'Bias',
                    6:'Tap',
                    7:'Puff',
                    8:'x',
                    9:'y',
                    10:'Morphwidth',
                    11:'Midline',
                    12:'Area',
                    13:'Angular Speed',
                    14:'Aspect Ratio',
                    15:'Kink',
                    16:'Curve',
                    17:'Crab',
                    18:'Pathlength'}, axis=1)
        
        #check function here for NaN Columns
        DF_Total['plate'] = 0

        print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

    return{
            'N': Strain_N,
            'Confirm':DF_Total
            # 'Final': DF_Final
    }


print('done step 5')

done step 5


# 6.1 Process Data
Gurmehak - new annotation here

In [26]:
genotype=[]
for f in filelist:
    genotype.append(f.split('/')[-3])

genotypes=np.unique(genotype).tolist()

if Screen =="Neuron_Genes_Screen":
    genotypes.insert(0, genotypes.pop(genotypes.index("N2_XJ1")))
    genotypes.insert(0, genotypes.pop(genotypes.index("N2_N2")))
else:
    genotypes.insert(0, genotypes.pop(genotypes.index("N2")))

nstrains = list(range(1, len(genotypes) + 1))
StrainNames = {nstrains[i]: genotypes[i] for i in range(len(nstrains))}

print(f"Number of genotypes/strains in the experiment: {len(genotypes)}")

# Display the first 5 Strain names in the experiment
for k in list(StrainNames)[:5]:
    print(f"{k}: {StrainNames[k]}")


print("done step 6.1")

# <---------------- Test element to use for dictionary buidling -------------------
# s = '/Users/Joseph/Desktop/OnFoodOffFoodTest/N2_OnFood/20220401_163048/N2_10x1_n96h20C_360sA0401_ka.00065.dat'
# slist=s.split('/')[5]
# print(slist)
# print(list(range(1,5+1)))

Number of genotypes/strains in the experiment: 3
1: N2
2: hipr-1_ok1081
3: hipr-1_tm10120
done step 6.1


# 6.2 Process Data (just run this cell)

In [30]:
DataLists = [0] # generates empty list at index 0 because we want indexing to start at 1 
                # when I say #1, I want the first point, not the second point

# the loop below goes through the dictionary in step 6.1 and processes data
for s in tqdm(StrainNames.values()): 
    if not s == '':
        DataLists.append(ProcessData(s)['Confirm']) # appends all data into a list of dataframes

print('done step 6.2')

  0%|          | 0/3 [00:00<?, ?it/s]

Strain N2
Number of plates: 5
File: /Users/gurmehak/Documents/RankinLab/Test_Datasets/PDScreen_TapHab_August15_2022/N2/20220815_101538/N2_10x2_f72h20C_600s31x10s10s_B0811ab.dat
File: /Users/gurmehak/Documents/RankinLab/Test_Datasets/PDScreen_TapHab_August15_2022/N2/20220815_102652/N2_10x2_f96h20C_600s31x10s10s_A0811aa.dat
File: /Users/gurmehak/Documents/RankinLab/Test_Datasets/PDScreen_TapHab_August15_2022/N2/20220815_122801/N2_10x2_f96h20C_600s31x10s10s_A0811ad.dat
File: /Users/gurmehak/Documents/RankinLab/Test_Datasets/PDScreen_TapHab_August15_2022/N2/20220815_121502/N2_10x2_f72h20C_600s31x10s10s_B0811ae.dat
File: /Users/gurmehak/Documents/RankinLab/Test_Datasets/PDScreen_TapHab_August15_2022/N2/20220815_103433/N2_10x2_f72h20C_600s31x10s10s_C0811ac.dat
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Strain hipr-1_ok1081
Number of plates: 4
File: /Users/gurmehak/

# Convert float64 data to float32 to reduce memory load (can also convert to 16 if needed)

For plain english:

float16 = 4 decimal points

float32 = 8 decimal points

float64 = 16 decimal points

more decimal points = more data/memory that computer has to keep track of

In [1]:
# commented out this section in case memory load needs to be reduced

# for n in tqdm(DataLists[1:]:
#     print(n)
#     TestData=n
#     TestData[TestData.select_dtypes(np.float64).columns]=TestData.select_dtypes(np.float64).astype(np.float16)
    

In [ ]:
DataLists[1].head()

    Time   n  Number  Instantaneous Speed  Interval Speed  Bias  Tap  Puff  \
0  0.009  13       0                  0.0             0.0   0.0    0     0   
1  0.061  13       0                  0.0             0.0   0.0    0     0   
2  0.089  12       0                  0.0             0.0   0.0    0     0   
3  0.136  12       0                  0.0             0.0   0.0    0     0   
4  0.168  12       0                  0.0             0.0   0.0    0     0   

     x    y  Morphwidth  Midline  Area  Angular Speed  Aspect Ratio  Kink  \
0  0.0  0.0         0.0      0.0   0.0            0.0           0.0   0.0   
1  0.0  0.0         0.0      0.0   0.0            0.0           0.0   0.0   
2  0.0  0.0         0.0      0.0   0.0            0.0           0.0   0.0   
3  0.0  0.0         0.0      0.0   0.0            0.0           0.0   0.0   
4  0.0  0.0         0.0      0.0   0.0            0.0           0.0   0.0   

   Curve  Crab  Pathlength Plate_id      Date                       

# 7. Grouping Data and Naming

For Gurmehak: I've removed outdated annotation here (user input is no longer required). Please annotate this section to better reflect what the below chunk does.

In [ ]:
TotalConcatenated=pd.concat(df.assign(dataset=StrainNames.get(i+1)) for i,df in enumerate(DataLists[1:]))

TotalConcatenated[['Gene', 'Allele']] = TotalConcatenated['dataset'].str.split('_', n=1, expand=True)

TotalConcatenated['Allele']=TotalConcatenated['Allele'].fillna('N2')

Baseline_data=TotalConcatenated[((TotalConcatenated.Time<=590)&(TotalConcatenated.Time >=490))] ### future changes to be made

Baseline_data=Baseline_data.drop(columns=["plate", "Tap", "Puff", "x","y"]).dropna().reset_index(drop=True)

print(Baseline_data.head())

Baseline_data['Screen']=Screen

Baseline_data.head()

   index     Time   n  Number  Instantaneous Speed  Interval Speed  Bias  \
0   9775  490.016  14      12               0.0823          0.1195  0.25   
1   9776  490.056  14      12               0.0736          0.1024  0.25   
2   9777  490.103  14      12               0.0784          0.1024  0.25   
3   9778  490.144  14      12               0.0970          0.1118  0.25   
4   9779  490.186  14      12               0.0994          0.1197  0.25   

   Morphwidth  Midline      Area  Angular Speed  Aspect Ratio  Kink  Curve  \
0      0.1078   1.0908  0.142641            6.1         0.225  31.1   25.0   
1      0.1059   1.0880  0.140150            5.3         0.219  30.9   24.7   
2      0.1050   1.0914  0.138935            5.2         0.219  31.0   24.6   
3      0.1054   1.0935  0.140575            5.8         0.218  30.6   24.5   
4      0.1110   1.1026  0.146894            5.6         0.227  31.4   24.4   

     Crab  Pathlength Plate_id      Date                         Screen  \

In [ ]:
print('done step 7')

# Save dataframe as `.csv`

In [34]:
Baseline_data.to_csv(f"{Baseline_data.Screen[0]}_baseline_output.csv")
print('saved as .csv!')

saved as .csv!


# Done!